In [87]:
from binarytree import Node
from collections import namedtuple

num_nodes = 15
global_node_list = [Node(chr(ord('A') + i)) for i in range(num_nodes)]

def print_nodes_list(view, nodes_list):
    print(f"{view}: {[(i, n.value) for i, n in enumerate(nodes_list)]}")

def nodes_list_to_map(l):
    l.sort(key=lambda x: x.value)
    return {i: n for i, n in enumerate(l)}

def init_node(node, nodes_list, layer):
    if node is None:
        return

    print_nodes_list(node.value, nodes_list)

    if layer == 0:
        return

    if len(nodes_list) == 0 or node.left is not None or node.right is not None:
        return

    nodes_list.sort(key=lambda x: x.value)

    i = nodes_list.index(node)
    li = int(i + len(nodes_list) * 1 / 3) % len(nodes_list)
    ri = int(i + len(nodes_list) * 2 / 3) % len(nodes_list)
    
    if i != li and (nodes_list[li].left is None and nodes_list[li].right is None):
        node.left = nodes_list[li]
    if i != ri and (nodes_list[ri].left is None and nodes_list[ri].right is None):
        node.right = nodes_list[ri]

    nodes_list = nodes_list[0:i] + nodes_list[i+1:]
    init_node(node.left, nodes_list.copy(), layer-1)
    init_node(node.right, nodes_list.copy(), layer-1)

print_nodes_list("global", global_node_list)

# originator_node = global_node_list[0]
# originator_node_list = global_node_list[1:]
# init_node(originator_node, originator_node_list, 3)
init_node(global_node_list[0], global_node_list, 3)

print(global_node_list[0])


global: [(0, 'A'), (1, 'B'), (2, 'C'), (3, 'D'), (4, 'E'), (5, 'F'), (6, 'G'), (7, 'H'), (8, 'I'), (9, 'J'), (10, 'K'), (11, 'L'), (12, 'M'), (13, 'N'), (14, 'O')]
A: [(0, 'A'), (1, 'B'), (2, 'C'), (3, 'D'), (4, 'E'), (5, 'F'), (6, 'G'), (7, 'H'), (8, 'I'), (9, 'J'), (10, 'K'), (11, 'L'), (12, 'M'), (13, 'N'), (14, 'O')]
F: [(0, 'B'), (1, 'C'), (2, 'D'), (3, 'E'), (4, 'F'), (5, 'G'), (6, 'H'), (7, 'I'), (8, 'J'), (9, 'K'), (10, 'L'), (11, 'M'), (12, 'N'), (13, 'O')]
J: [(0, 'B'), (1, 'C'), (2, 'D'), (3, 'E'), (4, 'G'), (5, 'H'), (6, 'I'), (7, 'J'), (8, 'K'), (9, 'L'), (10, 'M'), (11, 'N'), (12, 'O')]
N: [(0, 'B'), (1, 'C'), (2, 'D'), (3, 'E'), (4, 'G'), (5, 'H'), (6, 'I'), (7, 'K'), (8, 'L'), (9, 'M'), (10, 'N'), (11, 'O')]
D: [(0, 'B'), (1, 'C'), (2, 'D'), (3, 'E'), (4, 'G'), (5, 'H'), (6, 'I'), (7, 'K'), (8, 'L'), (9, 'M'), (10, 'N'), (11, 'O')]
O: [(0, 'B'), (1, 'C'), (2, 'D'), (3, 'E'), (4, 'G'), (5, 'H'), (6, 'I'), (7, 'J'), (8, 'K'), (9, 'L'), (10, 'M'), (11, 'N'), (12, 'O')]
E: 

In [297]:
# Assumption 1: Unique addresses that can be sorted lexicographically
# Assumption 2: Full view of the network (i.e. list to all nodes is available)

# Definition 1: Layer - opposite of tree depth (i.e. max # of layers = tree height)
# Definition 2: Global Address Book - list of all nodes in the network
# Definition 3: Partial Address Book - partial list of all nodes in the network (either due to algorithm or due to lack of information)

# Let X be the # of messages sent by each node (e.g. binary tree => 2; ternary tree => 3)
# Let Y be the target % coverage of of each node (e.g. 2/3 means node aims to propagate message to ~66% of the network)

import warnings
warnings.filterwarnings("ignore", category=DeprecationWarning) 

import random
import heapq
from binarytree import Node, tree, bst, heap, build
from collections import defaultdict

# Helpers
def get_subset(l, i1, i2):
    if i1 <= i2:
        return set(l[i1:i2 + 1])
    return set(l[i1:] + l[:i2])

# Global params for 1 simulation
global_counter_send = 0
global_set_reached = set()
global_map_received = defaultdict(int)

# Core logic
def prop(addr, book, depth):
    global global_counter_send
    global global_set_reached
    global global_map_received

    if addr == None:
        return

    global_set_reached.add(addr)
    global_map_received[addr] += 1

    print('-----')
    if len(book) <= 1:
        global_depth_counter[depth] += 1
        print(f"{addr} - {book}")
        print("No propagation")
        return

    n = len(book)
    i = book.index(addr)
    s = n * Y

    r1 = (int(i % n), int((i + s - 1) % n))
    r2 = (int((i + s) % n), int((i + s + s) % n))

    r1_set = get_subset(book, int(r1[0]), int(r1[1]))
    r2_set = get_subset(book, int(r2[0]), int(r2[1])) 

    r1_set.discard(addr)
    r2_set.discard(addr)

    bookp = set(book)
    t1, t2 = None, None

    if len(r1_set) > 0:
        t1 = random.sample(r1_set, 1)[0]
        bookp.discard(t1)
    if len(r2_set) > 0:
        t2 = random.sample(r2_set, 1)[0]
        bookp.discard(t2)

    # Only for visualization purposes
    heapq.heapify(book)
    root = build(book)
    
    print(root)
    print(f"{addr} - {book}")    
    print(f"R1: {r1} : {r1_set} => sending to {t1}")
    print(f"R2: {r2} : {r2_set} => sending to {t2}")

    if t1 is not None:
        global_counter_send += 1
        prop(t1, list(r1_set), depth + 1)
    
    if t2 is not None:
        global_counter_send += 1
        prop(t2, list(r2_set), depth + 1)
    
    # This is a demote (not a send) so we do not increment `global_counter_send`
    prop(addr, list(bookp), depth + 1)

def agg_dicts(d1, d2):
    return {k: d1.get(k, 0) + d2.get(k, 0) for k in set(d1) | set(d2)}

# Parameters
X = 2 # This is not parameterized but manually implemented. We use binary trees for simplicity.
Y = 2/3 # Coverage with each send (i.e. 66% of the network)
N = 6 # Numbers of nodes
nodes_addr = sorted([chr(ord('A') + i) for i in range(N)])

num_simulations = 1000
global_counter_send_acc = 0
global_map_received_acc = defaultdict(int)
global_depth_counter_acc = defaultdict(int)

for i in range(num_simulations):
    # Reset global params for simulation
    global_counter_send = 0
    global_set_reached = set()
    global_map_received = defaultdict(int)
    global_depth_counter = defaultdict(int)

    # Start simulation    
    prop(nodes_addr[0], nodes_addr, 0)
    
    # Print results
    print('###################')
    print(f"Target Coverage: {Y}")
    print(f"Num nodes: {num_nodes}")
    print(f"Global Send Counter: {global_counter_send}")
    print(f"Global Set Reached: {sorted(list(global_set_reached))}")
    print(f"Global # Times Received: {dict(dict(sorted(global_map_received.items(), key=lambda item: -item[1])))}")
    print(f"Nodes not reached: {global_set_reached.difference(nodes_addr)}")

    # Aggregate results
    global_counter_send_acc += global_counter_send
    global_map_received_acc = agg_dicts(global_map_received_acc, global_map_received)
    global_depth_counter_acc = agg_dicts(global_depth_counter_acc, global_depth_counter)

-----

    __A__
   /     \
  B       C
 / \     /
D   E   F

A - ['A', 'B', 'C', 'D', 'E', 'F']
R1: (0, 3) : {'D', 'B', 'C'} => sending to C
R2: (4, 2) : {'F', 'B', 'E'} => sending to B
-----

  B
 / \
D   C

C - ['B', 'D', 'C']
R1: (2, 0) : set() => sending to None
R2: (1, 0) : {'B'} => sending to B
-----
B - ['B']
No propagation
-----

  C
 /
D

C - ['C', 'D']
R1: (1, 1) : set() => sending to None
R2: (0, 1) : {'D'} => sending to D
-----
D - ['D']
No propagation
-----
C - ['C']
No propagation
-----

  B
 / \
F   E

B - ['B', 'F', 'E']
R1: (1, 2) : {'E'} => sending to E
R2: (0, 2) : {'F', 'E'} => sending to E
-----
E - ['E']
No propagation
-----

  E
 /
F

E - ['E', 'F']
R1: (1, 1) : set() => sending to None
R2: (0, 1) : {'F'} => sending to F
-----
F - ['F']
No propagation
-----
E - ['E']
No propagation
-----

  B
 /
F

B - ['B', 'F']
R1: (0, 0) : set() => sending to None
R2: (1, 0) : {'F'} => sending to F
-----
F - ['F']
No propagation
-----
B - ['B']
No propagation
-----

    A
   

In [340]:
# global_counter_send_acc
# global_map_received_acc

import plotly.figure_factory as ff
import pandas as pd
import numpy as np
import plotly.express as px
import plotly.graph_objects as go
from plotly.offline import plot
from plotly.subplots import make_subplots



## Depth distribution
counts_l = []
for d, c in global_depth_counter_acc.items():
    counts_l.extend([d] * c)
depths, counts = zip(*list(global_depth_counter_acc.items()))

counts_mean = sum(counts_l) / len(counts_l)
counts_variance = sum([((x - counts_mean) ** 2) for x in counts_l]) / len(counts_l)
counts_std = counts_variance ** 0.5
counts_normal_data = np.random.normal(counts_mean, counts_std, size=100) # replace with your own data source

## Send distribution
counts_send = []
for d, c in global_map_received_acc.items():
    counts_send.extend([d] * c)
nodes, counts_send = zip(*list(global_map_received_acc.items()))

counts_send_mean = sum(counts_send) / len(counts_send)
counts_send_variance = sum([((x - counts_send_mean) ** 2) for x in counts_send]) / len(counts_send)
counts_send_std = counts_send_variance ** 0.5
counts_send_normal_data = np.random.normal(counts_send_mean, counts_send_std, size=100) # replace with your own data source

# Prepare Figure
fig = make_subplots(
    rows=2, cols=1,
    specs=[
        [{"secondary_y": True}],
        [{"secondary_y": True}]],
    row_heights=[5, 5],
    subplot_titles=(
        f'Depth Distribution: Mean {round(counts_mean, 2)}; Std {round(counts_std, 2)}',
        f'Message Send Distribution: Mean {round(counts_send_mean, 2)}; Std {round(counts_send_std, 2)}',
    ))
fig.update_layout(
    height=800,
)

## Show Raw Data
fig.add_trace(
    go.Bar(x=depths, y=counts, name="raw counts", opacity=0.8),
    secondary_y=False,
)
fig.add_trace(
    go.Bar(x=nodes, y=counts_send, name="raw counts", opacity=0.8),
    secondary_y=False,
    row=2, col=1,
)

## Show Bars - Depth
rstd = counts_mean + counts_std
lstd = counts_mean - counts_std
ymax = 10000
fig.add_shape(type="line",x0=counts_mean, x1=counts_mean, y0 =0, y1=ymax , xref='x', yref='y',
               line = dict(color = 'blue', dash = 'dash'))
fig.add_shape(type="line",x0=rstd, x1=rstd, y0 =0, y1=ymax , xref='x', yref='y',
               line = dict(color = 'red', dash = 'dash'))
fig.add_shape(type="line",x0=lstd, x1=lstd, y0 =0, y1=ymax , xref='x', yref='y',
               line = dict(color = 'red', dash = 'dash'))

## Show Bars - Send
# rstd = counts_send_mean + counts_send_std
# lstd = counts_send_mean - counts_send_std
# ymax = 10000
# fig.add_shape(type="line",x0=counts_mean, x1=counts_mean, y0 =0, y1=ymax , xref='x', yref='y',
#                line = dict(color = 'blue', dash = 'dash'),
#                row=2, col=1)
# fig.add_shape(type="line",x0=rstd, x1=rstd, y0 =0, y1=ymax , xref='x', yref='y',
#                line = dict(color = 'red', dash = 'dash'),
#                row=2, col=1)
# fig.add_shape(type="line",x0=lstd, x1=lstd, y0 =0, y1=ymax , xref='x', yref='y',
#                line = dict(color = 'red', dash = 'dash'),
#                row=2, col=1)

## Show Normal Distribution - Depths
fig2 = ff.create_distplot([counts_normal_data], ['simulated normal distr'])
fig.add_trace(go.Histogram(fig2['data'][0], marker_color='orange', opacity=0.2), secondary_y=True)
fig.add_trace(go.Scatter(fig2['data'][1], line=dict(color='orange', width=0.5) ), secondary_y=True)
fig.add_trace(go.Scatter(fig2['data'][2], line=dict(color='orange', width=0.5)), secondary_y=True)

## Show Normal Distribution - Sends
# fig2 = ff.create_distplot([counts_send_normal_data], ['normal distr'])
# fig.add_trace(go.Histogram(fig2['data'][0], marker_color='orange', opacity=0.2), secondary_y=True,
#                row=2, col=1)
# fig.add_trace(go.Scatter(fig2['data'][1], line=dict(color='orange', width=0.5) ), secondary_y=True,
#                row=2, col=1)
# fig.add_trace(go.Scatter(fig2['data'][2], line=dict(color='orange', width=0.5)), secondary_y=True,
#                row=2, col=1)

## Print Results
print(f"Target Coverage: {round(Y, 3)}")
print(f"Num nodes: {num_nodes}")
print(f"Num simulations: {num_simulations}")
print(f"Avg # messages per simulation: {round(global_counter_send_acc / num_simulations, 2)}")
fig.show()

Target Coverage: 0.667
Num nodes: 6
Num simulations: 1000
Avg # messages per simulation: 12.76


In [334]:
global_map_received_acc

{'D': 3194, 'B': 3787, 'F': 3465, 'A': 4188, 'C': 3838, 'E': 3999}